# Phase 1, Flase Flag Data

## Training the model

In [1]:
import numpy as np
import pandas
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor,BaggingRegressor
URL = 'C:\Users\Kowshik\Desktop\Lighthouse\Consolidated data\MTV_T.xlsx'
df_train = pandas.read_excel(URL,sheetname=1)
df_total = pandas.read_excel(URL,sheetname=0)
df_test = pandas.read_excel(URL, sheetname=2)


In [55]:
df_train_false = df_train[:]
#df_train_false.Level= df_train_false.Level.astype(str)
#Dropping true Flag values
df_train_false = df_train_false[df_train_false.Level != True]

#deleting unwanted columns
                                
del df_train_false['EvType']
del df_train_false['PromoCategory']
del df_train_false['PromoType']
del df_train_false['EndTime']


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
for i in range(1,len(df_train_false['Description'].values)-1):
    priorProgram.append(df_train_false['Description'].values[i-1])
    subsequentProgram.append(df_train_false['Description'].values[i+1])
df_train_false =df_train_false.drop(df_train_false.index[[0]])
df_train_false =df_train_false.drop(df_train_false.index[[len(df_train_false['Description'].values)-1]])
df_train_false['priorProgram'] = priorProgram 
df_train_false['subsequentProgram'] =  subsequentProgram




#Categerizing the channels
df_PR = df_total[:]
df_PR = df_PR[df_PR.Level != True]
df_PR = df_PR[['Description','Impressions','ProgrammeTheme']]
Description = df_PR['Description'].values
for i in range(len(df_PR['Description'].values)):
    if df_PR['ProgrammeTheme'].values[i] == 'Movies / Plays':
         Description[i] = 'Movies'
    if '1 Hour Special' in df_PR['Description'].values[i]:
        Description[i] = df_PR['Description'].values[i].replace('- 1 Hour Special','').strip()  
df_PR['Description'] = Description
A = np.unique(df_PR['Description'].values)
PI = {}
for i in A:
    PI[i] = []
    for j in range(len(df_PR['Description'].values)):
        if df_PR['Description'].values[j] == i:
            PI[i].append(df_PR['Impressions'].values[j])
PI_AVG = {}
for i in A:
    PI_AVG[i] = np.mean(PI[i])
    
clus = []
for i in PI_AVG.values()[1:]:
    clus.append([i])
from sklearn.cluster import KMeans
clu = KMeans(n_clusters=6,n_init = 100,precompute_distances=False,n_jobs=1) #----------------- NO OF CLUSTERS ------------------>
clu.fit(clus)
Description_classified_prior = df_train_false['priorProgram'].values
for i in range(len(df_train_false['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_train_false['priorProgram'].values[i]])[0]
Description_classifiedsubsequent =df_train_false['subsequentProgram'].values
for i in range(len(df_train_false['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_train_false['subsequentProgram'].values[i]])[0]
df_train_false['priorProgram'] = Description_classified_prior
df_train_false['subsequentProgram'] = Description_classifiedsubsequent
df_train_false.priorProgram= df_train_false.priorProgram.astype(str) 
df_train_false.subsequentProgram= df_train_false.subsequentProgram.astype(str)



#Deleting and string conversion
df_train_false.Date = df_train_false.Date.astype(str)                    #Changing the Date as String 
df_train_false.ProgrammeTheme= df_train_false.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_train_false.ProgrammeGenre= df_train_false.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_train_false.StartTime= df_train_false.StartTime.astype(str) 
df_train_false.Length= df_train_false.Length.astype(str)

del df_train_false['Description']
del df_train_false['BreakCode']



#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_false['Length'].values]
df_train_false['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_train_false['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_train_false['Date'].values]
df_train_false['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_train_false['day'] = day           #------------------------------------------------------------------------------------Day--------->
#Weekday dealing
weekday = []
import datetime
for i in df_train_false['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_train_false['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_train_false['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
del df_train_false['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_train_false['StartTime'].values
for i in range(len(df_train_false['StartTime'].values)):
    if len(df_train_false['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_false['StartTime'].values[i].split()[1]    
df_train_false['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_train_false['hour'] = [map(int,i.split(':'))[0] for i in df_train_false['StartTime'].values]#--------------------hour------------------------------->
df_train_false.hour= df_train_false.hour.astype(str)
df_train_false['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_train_false['StartTime'].values]


Now Train Data is ready for training the model. encoding should be done with both testing and training data.

In [32]:
df_train_false


,ProgrammeTheme,ProgrammeGenre,StartTime,Level,Length,Impressions,Impressions1,priorProgram,subsequentProgram,Month,day,weekday,B_weekday,hour
10,Music,Film,6.000000,False,828,0.00,6.79,5,2,10,12,Monday,No,6
39,Music,Film,7.118611,False,2843,31.70,39.91,4,0,10,12,Monday,No,7
93,Music,Film,9.016944,False,2937,52.21,46.50,2,2,10,12,Monday,No,9
118,Music,Film,9.965833,False,7286,42.07,66.14,0,5,10,12,Monday,No,9
176,Reality,Real Life,11.999722,False,3017,14.03,76.26,2,1,10,12,Monday,No,11
199,Serials/Series - Fiction,Romance / Youth,13.016944,False,3453,16.38,39.03,5,2,10,12,Monday,No,13
219,Reality,Others,13.999722,False,3351,28.62,31.82,1,5,10,12,Monday,No,13
243,Reality,Real Life,15.016944,False,3026,28.04,92.53,2,2,10,12,Monday,No,15
293,Reality,Others,17.016944,False,3308,13.13,103.13,5,1,10,12,Monday,No,17
315,Serials/Series - Fiction,Romance / Youth,18.033889,False,1686,2.69,38.42,2,1,10,12,Monday,No,18


# #Testing the phase 1 model

In [56]:
df_test_false = df_test[:]
#Dropping true Flag values
df_test_false = df_test_false[df_test_false.Level != True]
#deleting unwanted columns                                  
del df_test_false['EvType']
del df_test_false['PromoCategory']
del df_test_false['PromoType']
del df_test_false['EndTime']
del df_test_false['Level'] 
del df_test_false['BreakCode']

#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
for i in range(1,len(df_test_false['Description'].values)-1):
    priorProgram.append(df_test_false['Description'].values[i-1])
    subsequentProgram.append(df_test_false['Description'].values[i+1])
df_test_false =df_test_false.drop(df_test_false.index[[0]])
df_test_false =df_test_false.drop(df_test_false.index[[len(df_test_false['Description'].values)-1]])
df_test_false['priorProgram'] = priorProgram 
df_test_false['subsequentProgram'] =  subsequentProgram

Description_classified_prior = df_test_false['priorProgram'].values
for i in range(len(df_test_false['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_test_false['priorProgram'].values[i]])[0]
Description_classifiedsubsequent =df_test_false['subsequentProgram'].values
for i in range(len(df_test_false['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_test_false['subsequentProgram'].values[i]])[0]
df_test_false['priorProgram'] = Description_classified_prior
df_test_false['subsequentProgram'] = Description_classifiedsubsequent
df_test_false.priorProgram= df_test_false.priorProgram.astype(str) 
df_test_false.subsequentProgram= df_test_false.subsequentProgram.astype(str)

#Deleting and string conversion
df_test_false.Date = df_test_false.Date.astype(str)                    #Changing the Date as String 
df_test_false.ProgrammeTheme= df_test_false.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_test_false.ProgrammeGenre= df_test_false.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_test_false.StartTime= df_test_false.StartTime.astype(str) 
df_test_false.Length= df_test_false.Length.astype(str)

del df_test_false['Description']


#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_false['Length'].values]
df_test_false['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_test_false['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_test_false['Date'].values]
df_test_false['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_test_false['day'] = day           #------------------------------------------------------------------------------------Day--------->
#Weekday dealing
weekday = []
import datetime
for i in df_test_false['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_test_false['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_test_false['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
del df_test_false['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_test_false['StartTime'].values
for i in range(len(df_test_false['StartTime'].values)):
    if len(df_test_false['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_false['StartTime'].values[i].split()[1]
df_test_false['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_test_false['hour'] = [map(int,i.split(':'))[0] for i in df_test_false['StartTime'].values]#--------------------hour------------------------------->
df_test_false.hour= df_test_false.hour.astype(str)
df_test_false['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_test_false['StartTime'].values]

In [51]:
df_test_false

,ProgrammeTheme,ProgrammeGenre,StartTime,Length,Impressions,Impressions1,priorProgram,subsequentProgram,Month,day,weekday,B_weekday,hour
5,Music,Film,6.000000,2481,5.44,9.42,1,2,5,2,Monday,No,6
27,Music,Film,7.186389,2931,24.91,40.92,4,2,5,2,Monday,No,7
51,Music,Film,8.186389,2953,38.27,69.21,2,0,5,2,Monday,No,8
62,Music,Film,9.016944,3057,51.24,72.15,2,2,5,2,Monday,No,9
84,Music,Film,10.000000,2962,30.37,59.42,0,2,5,2,Monday,No,10
106,Music,Film,10.982778,3553,22.62,44.57,2,5,5,2,Monday,No,10
124,Reality,Action/Thrill/Adventure,11.982778,3320,45.01,57.85,2,1,5,2,Monday,No,11
142,Serials/Series - Fiction,Romance / Youth,12.948889,1708,30.48,33.58,5,5,5,2,Monday,No,12
161,Reality,Action/Thrill/Adventure,13.542222,2928,28.22,37.22,1,1,5,2,Monday,No,13
177,Serials/Series - Fiction,Romance / Youth,14.491389,1704,9.34,28.46,5,5,5,2,Monday,No,14


In [57]:
#Decision channel
df_te = df_test_false[:]
Y_test_1 = np.array(df_test_false.Impressions)                       #Creating a array for impression Prediction 
del df_test_false['Impressions']
del df_test_false['Impressions1']
del df_te['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_te['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
#del df_te['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_te['Length']           #---------------------------------------------------------------------------------------------del length -------------->
#del df_te['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_te['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_te['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
#del df_te['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_te['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_te['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_te['Month'] #-------------------------------------------------------------------------------------------------Month------>
#df_te.StartTime= df_te.StartTime.astype(str)
df_te.day= df_te.day.astype(str)
df_te.Month= df_te.Month.astype(str)
del df_te['day']

,ProgrammeTheme,ProgrammeGenre,Length,Impressions,Impressions1,priorProgram,subsequentProgram,Month,day,weekday,hour
5,Music,Film,2481,5.44,9.42,1,2,5,2,Monday,6
27,Music,Film,2931,24.91,40.92,4,2,5,2,Monday,7
51,Music,Film,2953,38.27,69.21,2,0,5,2,Monday,8
62,Music,Film,3057,51.24,72.15,2,2,5,2,Monday,9
84,Music,Film,2962,30.37,59.42,0,2,5,2,Monday,10
106,Music,Film,3553,22.62,44.57,2,5,5,2,Monday,10
124,Reality,Action/Thrill/Adventure,3320,45.01,57.85,2,1,5,2,Monday,11
142,Serials/Series - Fiction,Romance / Youth,1708,30.48,33.58,5,5,5,2,Monday,12
161,Reality,Action/Thrill/Adventure,2928,28.22,37.22,1,1,5,2,Monday,13
177,Serials/Series - Fiction,Romance / Youth,1704,9.34,28.46,5,5,5,2,Monday,14


In [58]:
#Decision channel
Y_train_1 = np.array(df_train_false.Impressions)                       #Creating a array for impression Prediction 
del df_train_false['Impressions']
del df_train_false['Impressions1']
df_tr = df_train_false[:]
del df_tr['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_tr['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
#del df_tr['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_tr['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_tr['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_tr['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
#del df_tr['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
#del df_tr['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
del df_tr['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_tr['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_tr['Month'] #-------------------------------------------------------------------------------------------------Month------>
df_tr.day= df_tr.day.astype(str)
df_tr.Month= df_tr.Month.astype(str)

C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\core\generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [59]:
#ENCODING
X_train = df_tr.to_dict('records')
X_test = df_te.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te

In [60]:
#One Hot Encoding 
enc = DictVectorizer()
X_encoded_total =enc.fit_transform(X_total)
X_encoded_train =X_encoded_total[:len(X_tr)]
X_encoded_test =X_encoded_total[len(X_tr):]

In [61]:
#Extra Trees
reg1 = RandomForestRegressor()
reg1.fit(X_encoded_train,Y_train_1)
Y_pred1 = reg1.predict(X_encoded_test)
from sklearn.metrics import r2_score
r2_score(Y_test_1,Y_pred1)

0.20115241508937454

In [62]:
MAPE = []
ME = []
for i in range(len(Y_test_1)):
    ME.append(Y_pred1[i] - Y_test_1[i])
    if (Y_pred1[i] - Y_test_1[i]) == 0:
        MAPE.append(0)
    elif Y_test_1[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred1[i] - Y_test_1[i])/Y_test_1[i]))

In [63]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(Y_test_1,Y_pred1)
print mean_squared_error(Y_test_1,Y_pred1)**0.5
print r2_score(Y_test_1,Y_pred1)
print 'MAPE = ',sum(MAPE)/len(Y_test_1)
print sum(ME)/len(Y_test_1)


12.8725419847
18.1384464226
0.201152415089
MAPE =  3.00307965656
5.89122137405


In [268]:
len(Y_test_1)

898

In [64]:
df_te['pred'] = Y_pred1
df_te['act'] = Y_test_1

In [65]:
df = df_te[['pred','act']]


# Phase 2 program aggregate to program segment 

In [66]:
df


,pred,act
5,31.496,5.44
27,36.552,24.91
51,68.597,38.27
62,54.377,51.24
84,40.199,30.37
106,42.019,22.62
124,22.755,45.01
142,7.729,30.48
161,24.256,28.22
177,8.715,9.34


Training model


In [67]:
df_train_P2 = df_train[:]
#df_train_false.Level= df_train_false.Level.astype(str)
#Dropping true Flag values
df_train_P2 = df_train_P2[df_train_P2.EvType != 'Ad Break']
df_train_P2 = df_train_P2[df_train_P2.EvType != 'Promo']
df_train_P2 = df_train_P2[df_train_P2.ProgrammeTheme != 'Teleshopping']


df_train_P2.StartTime= df_train_P2.StartTime.astype(str)
df_train_P2.EndTime= df_train_P2.EndTime.astype(str)
df_train_P2.Level= df_train_P2.Level.astype(str)
#StartTime Dealing
Starttime = df_train_P2['StartTime'].values
for i in range(len(df_train_P2['StartTime'].values)):
    if len(df_train_P2['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_P2['StartTime'].values[i].split()[1]
df_train_P2['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
EndTime = df_train_P2['EndTime'].values
for i in range(len(df_train_P2['EndTime'].values)):
    if len(df_train_P2['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_train_P2['EndTime'].values[i].split()[1]
df_train_P2['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
for i in range(int(len(df_train_P2['Level'].values))):
    if df_train_P2['Level'].values[i] == 'True':
        for j in range(i,-1,-1):
            if df_train_P2['Level'].values[j] == 'False':
                priorProgram.append(df_train_P2['Impressions'].values[j])
                priorstarttime.append(df_train_P2['StartTime'].values[j])
                priorendtime.append(df_train_P2['EndTime'].values[j])
                break
                
for i in range(int(len(df_train_P2['Level'].values))):
    if df_train_P2['Level'].values[i] == 'True':
        for j in range(i,int(len(df_train_P2['EvType'].values))):
            if df_train_P2['Level'].values[j] == 'False':
                subsequentProgram.append(df_train_P2['Impressions'].values[j])
                subsequentstarttime.append(df_train_P2['StartTime'].values[j])
                subsequentendtime.append(df_train_P2['EndTime'].values[j])
                break

                
df_train_P2 = df_train_P2[df_train_P2.Level != 'False']
df_train_P2['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_train_P2['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_train_P2['priorstarttime'] = priorstarttime
df_train_P2['priorendtime'] =  priorendtime
df_train_P2['subsequentstarttime'] =  subsequentstarttime
df_train_P2['subsequentendtime'] =  subsequentendtime
df_train_P2  = df_train_P2[df_train_P2.priorProgram.notnull()]
df_train_P2  = df_train_P2[df_train_P2.subsequentProgram.notnull()]


df_train_P2.Length= df_train_P2.Length.astype(str)
df_train_P2.StartTime= df_train_P2.StartTime.astype(str)
df_train_P2.priorstarttime= df_train_P2.priorstarttime.astype(str)
df_train_P2.priorendtime= df_train_P2.priorendtime.astype(str)
df_train_P2.subsequentstarttime= df_train_P2.subsequentstarttime.astype(str)
df_train_P2.subsequentendtime= df_train_P2.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['Length'].values]
df_train_P2['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['StartTime'].values]
df_train_P2['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['EndTime'].values]
df_train_P2['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['priorstarttime'].values]
df_train_P2['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['priorendtime'].values]
df_train_P2['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['subsequentstarttime'].values]
df_train_P2['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P2['subsequentendtime'].values]
df_train_P2['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_train_P2['subsequentProgram'].values)):
    L_1.append(abs(df_train_P2['StartTime'].values[i] - df_train_P2['priorstarttime'].values[i]))
    L_2.append(abs(df_train_P2['priorendtime'].values[i] - df_train_P2['EndTime'].values[i]))
df_train_P2['L_1'] = L_1
df_train_P2['L_2'] = L_2

X_train_2 = []
Y_train_2 = []
for i in range(len(df_train_P2['priorProgram'].values)):
    X_train_2.append([df_train_P2['priorProgram'].values[i],df_train_P2['subsequentProgram'].values[i],df_train_P2['L_1'].values[i],df_train_P2['L_2'].values[i]])
    Y_train_2.append(df_train_P2['Impressions'].values[i])

Testing model

In [68]:
df_test_P2 = df_test[:]
#df_test_false.Level= df_test_false.Level.astype(str)
#Dropping true Flag values
df_test_P2 = df_test_P2[df_test_P2.EvType != 'Ad Break']
df_test_P2 = df_test_P2[df_test_P2.EvType != 'Promo']
df_test_P2 = df_test_P2[df_test_P2.ProgrammeTheme != 'Teleshopping']


df_test_P2.StartTime= df_test_P2.StartTime.astype(str)
df_test_P2.EndTime= df_test_P2.EndTime.astype(str)
df_test_P2.Level= df_test_P2.Level.astype(str)
#StartTime Dealing
Starttime = df_test_P2['StartTime'].values
for i in range(len(df_test_P2['StartTime'].values)):
    if len(df_test_P2['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_P2['StartTime'].values[i].split()[1]
df_test_P2['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
EndTime = df_test_P2['EndTime'].values
for i in range(len(df_test_P2['EndTime'].values)):
    if len(df_test_P2['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_test_P2['EndTime'].values[i].split()[1]
df_test_P2['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
for i in range(int(len(df_test_P2['Level'].values))):
    if df_test_P2['Level'].values[i] == 'True':
        for j in range(i,-1,-1):
            if df_test_P2['Level'].values[j] == 'False':
                priorProgram.append(df_test_P2['Impressions'].values[j])
                priorstarttime.append(df_test_P2['StartTime'].values[j])
                priorendtime.append(df_test_P2['EndTime'].values[j])
                break
                
for i in range(int(len(df_test_P2['Level'].values))):
    if df_test_P2['Level'].values[i] == 'True':
        for j in range(i,int(len(df_test_P2['EvType'].values))):
            if df_test_P2['Level'].values[j] == 'False':
                subsequentProgram.append(df_test_P2['Impressions'].values[j])
                subsequentstarttime.append(df_test_P2['StartTime'].values[j])
                subsequentendtime.append(df_test_P2['EndTime'].values[j])
                break

                
df_test_P2 = df_test_P2[df_test_P2.Level != 'False']
df_test_P2['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_test_P2['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_test_P2['priorstarttime'] = priorstarttime
df_test_P2['priorendtime'] =  priorendtime
df_test_P2['subsequentstarttime'] =  subsequentstarttime
df_test_P2['subsequentendtime'] =  subsequentendtime
df_test_P2  = df_test_P2[df_test_P2.priorProgram.notnull()]
df_test_P2  = df_test_P2[df_test_P2.subsequentProgram.notnull()]


df_test_P2.Length= df_test_P2.Length.astype(str)
df_test_P2.StartTime= df_test_P2.StartTime.astype(str)
df_test_P2.priorstarttime= df_test_P2.priorstarttime.astype(str)
df_test_P2.priorendtime= df_test_P2.priorendtime.astype(str)
df_test_P2.subsequentstarttime= df_test_P2.subsequentstarttime.astype(str)
df_test_P2.subsequentendtime= df_test_P2.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['Length'].values]
df_test_P2['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['StartTime'].values]
df_test_P2['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['EndTime'].values]
df_test_P2['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['priorstarttime'].values]
df_test_P2['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['priorendtime'].values]
df_test_P2['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['subsequentstarttime'].values]
df_test_P2['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P2['subsequentendtime'].values]
df_test_P2['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_test_P2['subsequentProgram'].values)):
    L_1.append(abs(df_test_P2['StartTime'].values[i] - df_test_P2['priorstarttime'].values[i]))
    L_2.append(abs(df_test_P2['priorendtime'].values[i] - df_test_P2['EndTime'].values[i]))
df_test_P2['L_1'] = L_1
df_test_P2['L_2'] = L_2

X_test_2 = []
Y_test_2 = []
for i in range(len(df_test_P2['priorProgram'].values)):
    X_test_2.append([df_test_P2['priorProgram'].values[i],df_test_P2['subsequentProgram'].values[i],df_test_P2['L_1'].values[i],df_test_P2['L_2'].values[i]])
    Y_test_2.append(df_test_P2['Impressions'].values[i])

In [69]:
from sklearn.linear_model import LinearRegression
reg2 = GradientBoostingRegressor()
reg2.fit(X_train_2,Y_train_2)
Y_pred_2 = reg2.predict(X_test_2)

In [70]:
from sklearn.metrics import r2_score
r2_score(Y_test_2,Y_pred_2)

0.62081284171953266

In [71]:
MAPE = []
for i in range(len(Y_test_2)):
    ME.append(Y_pred_2[i] - Y_test_2[i])
    if (Y_pred_2[i] - Y_test_2[i]) == 0:
        MAPE.append(0)
    elif Y_test_2[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred_2[i] - Y_test_2[i])/Y_test_2[i]))

In [72]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(Y_test_2,Y_pred_2)
print r2_score(Y_test_2,Y_pred_2)
print (mean_squared_error(Y_test_2,Y_pred_2))**0.5
print sum(MAPE)/len(Y_test_2)
print sum(ME)/len(Y_test_2)

9.49939545551
0.62081284172
15.1931704684
1.82749257717
2.18889335095


# Phase3, Program segment to promo level

## Training the data 

In [73]:
df_train_P3 = df_train[:]
df_train_P3 = df_train_P3[df_train_P3.Level != False]
df_train_P3 =  df_train_P3[df_train_P3.EvType != 'Ad Break']
df_train_P3 =  df_train_P3[df_train_P3.ProgrammeTheme != 'Teleshopping']

In [74]:

################################################################
df_train_P3.StartTime= df_train_P3.StartTime.astype(str)
df_train_P3.EndTime= df_train_P3.EndTime.astype(str)
#StartTime Dealing
Starttime = df_train_P3['StartTime'].values
for i in range(len(df_train_P3['StartTime'].values)):
    if len(df_train_P3['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_P3['StartTime'].values[i].split()[1]
df_train_P3['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_train_P3['EndTime'].values
for i in range(len(df_train_P3['EndTime'].values)):
    if len(df_train_P3['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_train_P3['EndTime'].values[i].split()[1]
df_train_P3['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_train_P3['EvType'].values))):
    if df_train_P3['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_train_P3['EvType'].values[j] == 'Program':
                priorProgram.append(df_train_P3['Impressions'].values[j])
                priorstarttime.append(df_train_P3['StartTime'].values[j])
                priorendtime.append(df_train_P3['EndTime'].values[j])
                break
for i in range(int(len(df_train_P3['EvType'].values))):
    if df_train_P3['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_train_P3['EvType'].values))):
            if df_train_P3['EvType'].values[j] == 'Program':
                subsequentProgram.append(df_train_P3['Impressions'].values[j])
                subsequentstarttime.append(df_train_P3['StartTime'].values[j])
                subsequentendtime.append(df_train_P3['EndTime'].values[j])
                break

                
df_train_P3 = df_train_P3[df_train_P3.EvType != 'Program']
df_train_P3['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_train_P3['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_train_P3['priorstarttime'] = priorstarttime
df_train_P3['priorendtime'] =  priorendtime
df_train_P3['subsequentstarttime'] =  subsequentstarttime
df_train_P3['subsequentendtime'] =  subsequentendtime
df_train_P3  = df_train_P3[df_train_P3.priorProgram.notnull()]
df_train_P3  = df_train_P3[df_train_P3.subsequentProgram.notnull()]  


df_train_P3.Length= df_train_P3.Length.astype(str)
df_train_P3.StartTime= df_train_P3.StartTime.astype(str)
df_train_P3.priorstarttime= df_train_P3.priorstarttime.astype(str)
df_train_P3.priorendtime= df_train_P3.priorendtime.astype(str)
df_train_P3.subsequentstarttime= df_train_P3.subsequentstarttime.astype(str)
df_train_P3.subsequentendtime= df_train_P3.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['Length'].values]
df_train_P3['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['StartTime'].values]
df_train_P3['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['EndTime'].values]
df_train_P3['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['priorstarttime'].values]
df_train_P3['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['priorendtime'].values]
df_train_P3['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['subsequentstarttime'].values]
df_train_P3['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_P3['subsequentendtime'].values]
df_train_P3['subsequentendtime'] = subsequentendtime_seconds




L_1 = []
L_2 = []
for i in range(len(df_train_P3['subsequentProgram'].values)):
    L_1.append(df_train_P3['StartTime'].values[i] - df_train_P3['priorendtime'].values[i])
    L_2.append(df_train_P3['subsequentstarttime'].values[i] - df_train_P3['EndTime'].values[i])
df_train_P3['L_1'] = L_1
df_train_P3['L_2'] = L_2



X_train_3 = []
Y_train_3 = []
for i in range(len(priorProgram)):
    X_train_3.append([df_train_P3['priorProgram'].values[i],df_train_P3['Length'].values[i],df_train_P3['subsequentProgram'].values[i],df_train_P3['L_1'].values[i],df_train_P3['L_2'].values[i]])
    Y_train_3.append(df_train_P3['Impressions'].values[i])

# #Testing

In [75]:
df_test_P3 = df_test[:]
df_test_P3 = df_test_P3[df_test_P3.Level != False]
df_test_P3 =  df_test_P3[df_test_P3.EvType != 'Ad Break']
df_test_P3 =  df_test_P3[df_test_P3.ProgrammeTheme != 'Teleshopping']

In [76]:

################################################################
df_test_P3.StartTime= df_test_P3.StartTime.astype(str)
df_test_P3.EndTime= df_test_P3.EndTime.astype(str)
#StartTime Dealing
Starttime = df_test_P3['StartTime'].values
for i in range(len(df_test_P3['StartTime'].values)):
    if len(df_test_P3['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_P3['StartTime'].values[i].split()[1]
df_test_P3['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_test_P3['EndTime'].values
for i in range(len(df_test_P3['EndTime'].values)):
    if len(df_test_P3['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_test_P3['EndTime'].values[i].split()[1]
df_test_P3['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_test_P3['EvType'].values))):
    if df_test_P3['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_test_P3['EvType'].values[j] == 'Program':
                priorProgram.append(df_test_P3['Impressions'].values[j])
                priorstarttime.append(df_test_P3['StartTime'].values[j])
                priorendtime.append(df_test_P3['EndTime'].values[j])
                break
for i in range(int(len(df_test_P3['EvType'].values))):
    if df_test_P3['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_test_P3['EvType'].values))):
            if df_test_P3['EvType'].values[j] == 'Program':
                subsequentProgram.append(df_test_P3['Impressions'].values[j])
                subsequentstarttime.append(df_test_P3['StartTime'].values[j])
                subsequentendtime.append(df_test_P3['EndTime'].values[j])
                break

                
df_test_P3 = df_test_P3[df_test_P3.EvType != 'Program']
df_test_P3['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_test_P3['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_test_P3['priorstarttime'] = priorstarttime
df_test_P3['priorendtime'] =  priorendtime
df_test_P3['subsequentstarttime'] =  subsequentstarttime
df_test_P3['subsequentendtime'] =  subsequentendtime
df_test_P3  = df_test_P3[df_test_P3.priorProgram.notnull()]
df_test_P3  = df_test_P3[df_test_P3.subsequentProgram.notnull()]  


df_test_P3.Length= df_test_P3.Length.astype(str)
df_test_P3.StartTime= df_test_P3.StartTime.astype(str)
df_test_P3.priorstarttime= df_test_P3.priorstarttime.astype(str)
df_test_P3.priorendtime= df_test_P3.priorendtime.astype(str)
df_test_P3.subsequentstarttime= df_test_P3.subsequentstarttime.astype(str)
df_test_P3.subsequentendtime= df_test_P3.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['Length'].values]
df_test_P3['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['StartTime'].values]
df_test_P3['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['EndTime'].values]
df_test_P3['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['priorstarttime'].values]
df_test_P3['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['priorendtime'].values]
df_test_P3['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['subsequentstarttime'].values]
df_test_P3['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_P3['subsequentendtime'].values]
df_test_P3['subsequentendtime'] = subsequentendtime_seconds




L_1 = []
L_2 = []
for i in range(len(df_test_P3['subsequentProgram'].values)):
    L_1.append(df_test_P3['StartTime'].values[i] - df_test_P3['priorendtime'].values[i])
    L_2.append(df_test_P3['subsequentstarttime'].values[i] - df_test_P3['EndTime'].values[i])
df_test_P3['L_1'] = L_1
df_test_P3['L_2'] = L_2



X_test_3 = []
Y_test_3 = []
for i in range(len(priorProgram)):
    X_test_3.append([df_test_P3['priorProgram'].values[i],df_test_P3['Length'].values[i],df_test_P3['subsequentProgram'].values[i],df_test_P3['L_1'].values[i],df_test_P3['L_2'].values[i]])
    Y_test_3.append(df_test_P3['Impressions'].values[i])

In [77]:
from sklearn.linear_model import LinearRegression
reg = GradientBoostingRegressor()
reg.fit(X_train_3,Y_train_3)
Y_pred_3 =  reg.predict(X_test_3)

In [78]:
from sklearn.metrics import r2_score
r2_score(Y_test_3,Y_pred_3)

0.48979083538296386

In [224]:
MAPE = []
ME = []
for i in range(len(Y_test_3)):
    ME.append(Y_pred_3[i] - Y_test_3[i])
    if (Y_pred_3[i] - Y_test_3[i]) == 0:
        MAPE.append(0)
    elif Y_test_3[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred_3[i] - Y_test_3[i])/Y_test_3[i]))

In [225]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(Y_test_3,Y_pred_3)
print mean_squared_error(Y_test_3,Y_pred_3)
print r2_score(Y_test_3,Y_pred_3)
print (mean_squared_error(Y_test_3,Y_pred_3))**0.5
print sum(MAPE)/len(Y_test_3)
print sum(ME)/len(Y_test_3)

174.185445713
111028.360982
0.967310116912
333.20918502
0.664523383743
26.5778523138


## bypass Phase Analysis 

In [226]:
df_train_bp = df_train[:]
df_train_bp =  df_train_bp[df_train_bp.EvType != 'Ad Break']
df_train_bp =  df_train_bp[df_train_bp.ProgrammeTheme != 'Teleshopping']
################################################################
df_train_bp.StartTime= df_train_bp.StartTime.astype(str)
df_train_bp.EndTime=   df_train_bp.EndTime.astype(str)
#StartTime Dealing
Starttime = df_train_bp['StartTime'].values
for i in range(len(df_train_bp['StartTime'].values)):
    if len(df_train_bp['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_train_bp['StartTime'].values[i].split()[1]
df_train_bp['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_train_bp['EndTime'].values
for i in range(len(df_train_bp['EndTime'].values)):
    if len(df_train_bp['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_train_bp['EndTime'].values[i].split()[1]
df_train_bp['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_train_bp['EvType'].values))):
    if df_train_bp['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_train_bp['Level'].values[j] == False:
                priorProgram.append(df_train_bp['Impressions'].values[j])
                priorstarttime.append(df_train_bp['StartTime'].values[j])
                priorendtime.append(df_train_bp['EndTime'].values[j])
                break
for i in range(int(len(df_train_bp['EvType'].values))):
    if df_train_bp['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_train_bp['EvType'].values))):
            if df_train_bp['Level'].values[j] == False:
                subsequentProgram.append(df_train_bp['Impressions'].values[j])
                subsequentstarttime.append(df_train_bp['StartTime'].values[j])
                subsequentendtime.append(df_train_bp['EndTime'].values[j])
                break

                
df_train_bp =  df_train_bp[df_train_bp.EvType != 'Program']
df_train_bp['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_train_bp['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_train_bp['priorstarttime'] = priorstarttime
df_train_bp['priorendtime'] =  priorendtime
df_train_bp['subsequentstarttime'] =  subsequentstarttime
df_train_bp['subsequentendtime'] =  subsequentendtime
df_train_bp  = df_train_bp[df_train_bp.priorProgram.notnull()]
df_train_bp  = df_train_bp[df_train_bp.subsequentProgram.notnull()]  


df_train_bp.Length= df_train_bp.Length.astype(str)
df_train_bp.StartTime= df_train_bp.StartTime.astype(str)
df_train_bp.priorstarttime= df_train_bp.priorstarttime.astype(str)
df_train_bp.priorendtime= df_train_bp.priorendtime.astype(str)
df_train_bp.subsequentstarttime= df_train_bp.subsequentstarttime.astype(str)
df_train_bp.subsequentendtime= df_train_bp.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['Length'].values]
df_train_bp['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['StartTime'].values]
df_train_bp['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['EndTime'].values]
df_train_bp['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['priorstarttime'].values]
df_train_bp['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['priorendtime'].values]
df_train_bp['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['subsequentstarttime'].values]
df_train_bp['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_train_bp['subsequentendtime'].values]
df_train_bp['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_train_bp['subsequentProgram'].values)):
    L_1.append(abs(df_train_bp['StartTime'].values[i] - df_train_bp['priorstarttime'].values[i]))
    L_2.append(abs(df_train_bp['priorendtime'].values[i] - df_train_bp['EndTime'].values[i]))
df_train_bp['L_1'] = L_1
df_train_bp['L_2'] = L_2

X_train_bp = []
Y_train_bp = []
for i in range(len(priorProgram)):
    X_train_bp.append([df_train_bp['priorProgram'].values[i],df_train_bp['subsequentProgram'].values[i],df_train_bp['L_1'].values[i],df_train_bp['L_2'].values[i]])
    Y_train_bp.append(df_train_bp['Impressions'].values[i])


In [227]:
########### TESTING

In [228]:
df_test_bp = df_test[:]
df_test_bp =  df_test_bp[df_test_bp.EvType != 'Ad Break']
df_test_bp =  df_test_bp[df_test_bp.ProgrammeTheme != 'Teleshopping']
################################################################
df_test_bp.StartTime= df_test_bp.StartTime.astype(str)
df_test_bp.EndTime=   df_test_bp.EndTime.astype(str)
#StartTime Dealing
Starttime = df_test_bp['StartTime'].values
for i in range(len(df_test_bp['StartTime'].values)):
    if len(df_test_bp['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_bp['StartTime'].values[i].split()[1]
df_test_bp['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df_test_bp['EndTime'].values
for i in range(len(df_test_bp['EndTime'].values)):
    if len(df_test_bp['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df_test_bp['EndTime'].values[i].split()[1]
df_test_bp['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
#df_CH['EvType'].values
for i in range(int(len(df_test_bp['EvType'].values))):
    if df_test_bp['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df_test_bp['Level'].values[j] == False:
                priorProgram.append(df_test_bp['Impressions'].values[j])
                priorstarttime.append(df_test_bp['StartTime'].values[j])
                priorendtime.append(df_test_bp['EndTime'].values[j])
                break
for i in range(int(len(df_test_bp['EvType'].values))):
    if df_test_bp['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_test_bp['EvType'].values))):
            if df_test_bp['Level'].values[j] == False:
                subsequentProgram.append(df_test_bp['Impressions'].values[j])
                subsequentstarttime.append(df_test_bp['StartTime'].values[j])
                subsequentendtime.append(df_test_bp['EndTime'].values[j])
                break

                
df_test_bp =  df_test_bp[df_test_bp.EvType != 'Program']
df_test_bp['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_test_bp['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_test_bp['priorstarttime'] = priorstarttime
df_test_bp['priorendtime'] =  priorendtime
df_test_bp['subsequentstarttime'] =  subsequentstarttime
df_test_bp['subsequentendtime'] =  subsequentendtime
df_test_bp  = df_test_bp[df_test_bp.priorProgram.notnull()]
df_test_bp  = df_test_bp[df_test_bp.subsequentProgram.notnull()]  


df_test_bp.Length= df_test_bp.Length.astype(str)
df_test_bp.StartTime= df_test_bp.StartTime.astype(str)
df_test_bp.priorstarttime= df_test_bp.priorstarttime.astype(str)
df_test_bp.priorendtime= df_test_bp.priorendtime.astype(str)
df_test_bp.subsequentstarttime= df_test_bp.subsequentstarttime.astype(str)
df_test_bp.subsequentendtime= df_test_bp.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['Length'].values]
df_test_bp['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['StartTime'].values]
df_test_bp['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['EndTime'].values]
df_test_bp['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['priorstarttime'].values]
df_test_bp['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['priorendtime'].values]
df_test_bp['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['subsequentstarttime'].values]
df_test_bp['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_bp['subsequentendtime'].values]
df_test_bp['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df_test_bp['subsequentProgram'].values)):
    L_1.append(abs(df_test_bp['StartTime'].values[i] - df_test_bp['priorstarttime'].values[i]))
    L_2.append(abs(df_test_bp['priorendtime'].values[i] - df_test_bp['EndTime'].values[i]))
df_test_bp['L_1'] = L_1
df_test_bp['L_2'] = L_2

X_test_bp = []
Y_test_bp = []
for i in range(len(priorProgram)):
    X_test_bp.append([df_test_bp['priorProgram'].values[i],df_test_bp['subsequentProgram'].values[i],df_test_bp['L_1'].values[i],df_test_bp['L_2'].values[i]])
    Y_test_bp.append(df_test_bp['Impressions'].values[i])


In [229]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train_bp,Y_train_bp)
Y_pred_bp =  reg.predict(X_test_bp)

In [230]:
from sklearn.metrics import r2_score
r2_score(Y_test_bp,Y_pred_bp)

0.90326795164143303

In [231]:
# All phase analysis
df = df_test
df['Numbers'] = np.arange(len(df))
df = df[df.ProgrammeTheme != 'Teleshopping']
df = df[df.EvType != 'Ad Break']
#del df['Impressions']

In [232]:
df.StartTime= df.StartTime.astype(str)
df.EndTime=   df.EndTime.astype(str)
#StartTime Dealing
Starttime = df['StartTime'].values
for i in range(len(df['StartTime'].values)):
    if len(df['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df['StartTime'].values[i].split()[1]
df['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time

EndTime = df['EndTime'].values
for i in range(len(df['EndTime'].values)):
    if len(df['EndTime'].values[i].split()) == 2:     
        EndTime[i] =  df['EndTime'].values[i].split()[1]
df['EndTime'] = EndTime #------------------------------------------------------------------------------------------Start time


In [233]:
#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
priorstarttime = []
priorProgramnumber = []
subsequentProgramnumber = []
priorendtime = []
subsequentstarttime = []
subsequentendtime = []
for i in range(int(len(df['EvType'].values))):
    if df['EvType'].values[i] == 'Promo':
        for j in range(i,-1,-1):
            if df['Level'].values[j] == False:
                priorProgram.append(df['Description'].values[j])
                priorProgramnumber.append(df['Numbers'].values[j])
                priorstarttime.append(df['StartTime'].values[j])
                priorendtime.append(df['EndTime'].values[j])
                break
for i in range(int(len(df['EvType'].values))):
    if df['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df['EvType'].values))):
            if df['Level'].values[j] == False:
                subsequentProgram.append(df['Description'].values[j])
                subsequentProgramnumber.append(df['Numbers'].values[j])
                subsequentstarttime.append(df['StartTime'].values[j])
                subsequentendtime.append(df['EndTime'].values[j])
                break

In [234]:
df =  df[df.EvType != 'Program']
df['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df['priorstarttime'] = priorstarttime
df['priorendtime'] =  priorendtime
df['subsequentstarttime'] =  subsequentstarttime
df['subsequentendtime'] =  subsequentendtime
df['subsequentProgramnumber'] = subsequentProgramnumber
df['priorProgramnumber'] = priorProgramnumber
df  = df[df.priorProgram.notnull()]
df  = df[df.subsequentProgram.notnull()]  

In [235]:

df.Length= df.Length.astype(str)
df.StartTime= df.StartTime.astype(str)
df.priorstarttime= df.priorstarttime.astype(str)
df.priorendtime= df.priorendtime.astype(str)
df.subsequentstarttime= df.subsequentstarttime.astype(str)
df.subsequentendtime= df.subsequentendtime.astype(str)
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['Length'].values]
df['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
StartTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['StartTime'].values]
df['StartTime'] = StartTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
EndTime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['EndTime'].values]
df['EndTime'] = EndTime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['priorstarttime'].values]
df['priorstarttime'] = priorstarttime_seconds#----------------------------------------------------------------------------------start time in Seconds----------------->
priorendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['priorendtime'].values]
df['priorendtime'] = priorendtime_seconds
subsequentstarttime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['subsequentstarttime'].values]
df['subsequentstarttime'] = subsequentstarttime_seconds
subsequentendtime_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df['subsequentendtime'].values]
df['subsequentendtime'] = subsequentendtime_seconds

L_1 = []
L_2 = []
for i in range(len(df['subsequentProgram'].values)):
    L_1.append(abs(df['StartTime'].values[i] - df['priorstarttime'].values[i]))
    L_2.append(abs(df['priorendtime'].values[i] - df['EndTime'].values[i]))
df['L_1'] = L_1
df['L_2'] = L_2

In [237]:
df_test_false = df_test[:]
#df_test_false.Level= df_test_false.Level.astype(str)
#Dropping true Flag values
df_test_false['Numbers'] = np.arange(len(df_test_false))
df_test_false = df_test_false[df_test_false.Level != True]
df_test_false = df_test_false[df_test_false.ProgrammeTheme != 'Teleshopping']
#deleting unwanted columns
                               
del df_test_false['EvType']
del df_test_false['PromoCategory']
del df_test_false['PromoType']
del df_test_false['EndTime']
del df_test_false['Level'] 
#Taggin 'Movies / Plays' into Movie
Description = df_test_false['Description'].values
for i in range(len(df_test_false['Description'].values)):
    if df_test_false['ProgrammeTheme'].values[i] == 'Movies / Plays':
         Description[i] = 'Movies'
    if '1 Hour Special' in df_test_false['Description'].values[i]:
        Description[i] = df_test_false['Description'].values[i].replace('- 1 Hour Special','').strip()       
df_test_false['Description'] = Description

#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
for i in range(1,len(df_test_false['Description'].values)-1):
    priorProgram.append(df_test_false['Description'].values[i-1])
    subsequentProgram.append(df_test_false['Description'].values[i+1])
df_test_false =df_test_false.drop(df_test_false.index[[0]])
df_test_false =df_test_false.drop(df_test_false.index[[len(df_test_false['Description'].values)-1]])
df_test_false['priorProgram'] = priorProgram 
df_test_false['subsequentProgram'] =  subsequentProgram

Description_classified_prior = df_test_false['priorProgram'].values
for i in range(len(df_test_false['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_test_false['priorProgram'].values[i]])[0]
Description_classifiedsubsequent =df_test_false['subsequentProgram'].values
for i in range(len(df_test_false['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_test_false['subsequentProgram'].values[i]])[0]
df_test_false['priorProgram'] = Description_classified_prior
df_test_false['subsequentProgram'] = Description_classifiedsubsequent
df_test_false.priorProgram= df_test_false.priorProgram.astype(str) 
df_test_false.subsequentProgram= df_test_false.subsequentProgram.astype(str)

#Deleting and string conversion
df_test_false.Date = df_test_false.Date.astype(str)                    #Changing the Date as String 
df_test_false.ProgrammeTheme= df_test_false.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_test_false.ProgrammeGenre= df_test_false.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_test_false.StartTime= df_test_false.StartTime.astype(str) 
df_test_false.Length= df_test_false.Length.astype(str)
Y_test_1 = np.array(df_test_false.Impressions)                       #Creating a array for impression Prediction 
del df_test_false['Impressions']
del df_test_false['Description']


#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_test_false['Length'].values]
df_test_false['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_test_false['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_test_false['Date'].values]
df_test_false['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_test_false['day'] = day           #------------------------------------------------------------------------------------Day--------->
#Weekday dealing
weekday = []
import datetime
for i in df_test_false['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_test_false['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_test_false['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
del df_test_false['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_test_false['StartTime'].values
for i in range(len(df_test_false['StartTime'].values)):
    if len(df_test_false['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_test_false['StartTime'].values[i].split()[1]
df_test_false['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_test_false['hour'] = [map(int,i.split(':'))[0] for i in df_test_false['StartTime'].values]#--------------------hour------------------------------->
df_test_false.hour= df_test_false.hour.astype(str)
df_test_false['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_test_false['StartTime'].values]


In [238]:
#Decision channel
df_tr = df_train_false[:]

#del df_tr['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_tr['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
#del df_tr['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_tr['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_tr['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_tr['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_tr['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
del df_tr['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_tr['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_tr['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_tr['Month'] #-------------------------------------------------------------------------------------------------Month------>

In [239]:
#Decision channel
df_te = df_test_false[:]
del df_te['Numbers']
#del df_te['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
#del df_te['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
del df_te['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_te['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_te['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_te['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_te['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
del df_te['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_te['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_te['day'] #-------------------------------------------------------------------------- ------------------------day------->
#del df_te['Month'] #-------------------------------------------------------------------------------------------------Month------>

In [240]:
#ENCODING
X_train = df_tr.to_dict('records')
X_test = df_te.to_dict('records')
X_tr = []
X_te = []
X_tr.extend(X_train)
X_te.extend(X_test)
X_total = X_tr + X_te

In [241]:
#Extra Trees
reg1 = ExtraTreesRegressor()
reg1.fit(X_encoded_train,Y_train_1)
Y_pred1 = reg1.predict(X_encoded_test)
from sklearn.metrics import r2_score
r2_score(Y_pred1 ,Y_test_1)

0.58619725214837648

In [242]:
df_test_false['predicted'] = Y_pred1
df_false = df_test_false[['predicted']]

In [243]:
df = df[df.priorProgramnumber != 0]
df = df[df.subsequentProgramnumber != 12372]

In [244]:
priorProgram = []
subsequentProgram = []
for i in df['priorProgramnumber'].values:
    priorProgram.append(df_false.to_dict()['predicted'][i])
    subsequentProgram.append(df_false.to_dict()['predicted'][i])
df['priorProgram'] = priorProgram
df['subsequentProgram'] = subsequentProgram

In [245]:
X_test_ca = []
for i in range(len(priorProgram)):
    X_test_ca.append([df['priorProgram'].values[i],df['subsequentProgram'].values[i],df['L_1'].values[i],df['L_2'].values[i]])

In [246]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train_bp,Y_train_bp)
Y_pred_ca =  reg.predict(X_test_ca)

In [247]:
r2_score(Y_pred_ca,df['Impressions'].values)

0.58163568964734291

In [251]:
import numpy as np
import matplotlib.pyplot as plt
#OUR DATA
A = 600
X = np.arange(A)
Y = df['Impressions'].values[:A]
x = np.arange(A)
y = Y_pred_ca[:A]
####################
#Visuvalization
canvas = plt.figure()
rect = canvas.patch
rect.set_facecolor('white')
sp1 = canvas.add_subplot(1,1,1, axisbg='w')
sp1.plot(x, y, 'Red', linewidth=2)
sp1.plot(X, Y, 'Black', linewidth=1)

sp1.patch.set_facecolor('white') #------> background colour

sp1.tick_params(axis='x', colors='green') #---------> X-value colour
sp1.tick_params(axis='y', colors='red') #----------> y-value colour

sp1.spines['bottom'].set_color('black')        #Colour of spines 
sp1.spines['top'].set_visible('False')
sp1.spines['left'].set_color('black')
sp1.spines['right'].set_visible('False')

sp1.set_title('Impressions vs time Using Extra Classifier', color='red')
sp1.set_xlabel('Time,BLUE -Actual', color='red')      #Colour of title and lables 
sp1.set_ylabel('Impression', color='red')
plt.plot(X,Y)
plt.plot(x,y)

plt.tight_layout()
plt.show()

In [269]:
df['Predicted Values'] = Y_pred_ca

In [252]:
MAPE = []
ME = []
for i in range(len(df['Impressions'].values)):
    ME.append(Y_pred_ca[i] - df['Impressions'].values[i])
    if (Y_pred_ca[i] - df['Impressions'].values[i]) == 0:
        MAPE.append(0)
    elif df['Impressions'].values[i] == 0:
        MAPE.append(1)
    else:
        MAPE.append(abs((Y_pred_ca[i] - df['Impressions'].values[i])/df['Impressions'].values[i]))

In [253]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
print mean_absolute_error(df['Impressions'].values,Y_pred_ca)
print mean_squared_error(df['Impressions'].values,Y_pred_ca)
print r2_score(df['Impressions'].values,Y_pred_ca)
print (mean_squared_error(df['Impressions'].values,Y_pred_ca))**0.5
print sum(MAPE)/len(df['Impressions'].values)
print sum(ME)/len(df['Impressions'].values)

661.502350736
2066069.11176
0.392256395027
1437.38272974
1.09526676166
68.9577149049
